# Twitter Sentiment Analysis - 04 Modeling

The stock market is a focus for investors to maximize their potential profits and consequently, the interest shown from the technical and financial sides in stock market prediction is always on the rise. However, stock market prediction is a problem known for its challenging nature due to its dependency on diverse factors that affect the market, these factors are unpredictable and cannot be taken into consideration such as political variables, and social media effects such as twitter on the stock market.

In this final part of this project, we will combine the stock data and its features, with vectorized representation of the tweets for the month of December 2022 to predict whether or not the adjusted closing price at the end of a trading-day is greater than or less than the previous trading-day. Models run are KNN, Logistic Regression, Decision Tree, Random Forest. SVM and ANN.

**Link(s) to previous notebook(s)**: \
00_Historical_Data_2014: https://github.com/parisvu07/Springboard_Data_Science/tree/main/Capstone_2_Twitter_Sentiment_Analysis \
01_Data_Wrangling:
https://github.com/parisvu07/Springboard_Data_Science/blob/main/Capstone_2_Twitter_Sentiment_Analysis/01_Data_Wrangling.ipynb \
02_Exploratory_Data_Analysis: https://github.com/parisvu07/Springboard_Data_Science/blob/main/Capstone_2_Twitter_Sentiment_Analysis/02_Exploratory_Data_Analysis.ipynb \
03_Preprocessing_and_Training_Data: https://github.com/parisvu07/Springboard_Data_Science/blob/main/Capstone_2_Twitter_Sentiment_Analysis/03_Preprocessing_and_Training_Data.ipynb

Quick fix for "Unable to render rich display": copy and paste the notebook link to https://nbviewer.org

## 4.1 Importing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#ignore warning messages to ensure clean outputs
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import gensim
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Word2Vec
LabeledSentence = gensim.models.doc2vec.TaggedDocument
from tqdm import tqdm
import string
import spacy
np.random.seed(42)

In [2]:
#Importing stock data from notebook "02_Exploratory_Data_Analysis"
stock_data = pd.read_csv('/Users/user/Documents/Springboard_Data_Science/Capstone_2_Twitter_Sentiment_Analysis/Data/03_stock_data.csv', encoding='latin-1')
stock_data = eda_stock_data.set_index('Dates')
stock_data = eda_stock_data.drop('Time', axis=1)
stock_data.head()

,Adj Close,Volume,%_change_Open,%_change_High,%_change_Low,%_change_Close,%_change_Volume
Dates,,,,,,,
2022-12-01,148.309998,71250400,NaN,NaN,NaN,NaN,NaN
2022-12-02,147.809998,65447400,-1.518116,-0.757731,-0.654803,-0.337132,-8.144516
2022-12-05,146.630005,68826400,1.240064,1.972972,0.082396,-0.798317,5.162925
2022-12-06,142.910004,64727200,-0.473707,-2.398619,-2.641151,-2.536999,-5.955854
2022-12-07,140.940002,69721100,-3.318151,-2.668030,-1.352874,-1.378491,7.715304


In [3]:
#Importing tweet data from previous notebook "03_Preprocessing_and_Training_Data"
trading_hours_tweets = pd.read_csv('/Users/user/Documents/Springboard_Data_Science/Capstone_2_Twitter_Sentiment_Analysis/Data/03_tweets_data.csv', lineterminator='\n')
trading_hours_tweets.head()

,Dates,Time,user,likes,source,text,Subjectivity,Polarity,Analysis,Sentiment,...,mention_count,punct_count,avg_wordlength,avg_sentlength,unique_vs_words,stopwords_vs_words,clean_text,tokens,tweet_without_stopwords,tweet_lemmatized
0,2022-12-30,20:29:43,LlcBillionaire,0,Twitter Web App,10 New Yearâs food traditions around the world,0.454545,0.136364,Positive,1.0,...,0,"{'! count': 0, '"" count': 0, '# count': 0, '$ ...",6.125000,8.0,1.000000,0.125000,new yearâ food tradition around the world,"['new', 'yearâ', 'food', 'tradition', 'around'...",new yearâ food tradition around world,"['new', 'yearâ\x80\x99', 'food', 'tradition', ..."
1,2022-12-30,20:29:32,skitontop1,0,Twitter Web App,Entries &amp; exits Daily! \nDiscord link belo...,0.500000,0.300000,Positive,1.0,...,0,"{'! count': 1, '"" count': 0, '# count': 0, '$ ...",7.428571,3.5,1.000000,0.000000,entrie amp exit daily \ndi cord link belo...,"['entrie', 'amp', 'exit', 'daily', 'di', 'cord...",entrie amp exit daily di cord link belowð,"['entrie', 'amp', 'exit', 'daily', 'di', 'cord..."
2,2022-12-30,20:29:28,StockJobberOG,0,Twitter Web App,$AAPL $MSFT $SPY $TSLA $AMZN $BRK.B\n\n,0.000000,0.000000,Neutral,0.0,...,0,"{'! count': 0, '"" count': 0, '# count': 0, '$ ...",6.166667,6.0,1.000000,0.000000,aapl m ft py t la amzn brk b\n\n,"['aapl', 'm', 'ft', 'py', 't', 'la', 'amzn', '...",aapl ft py la amzn brk b,"['aapl', 'ft', 'py', 'la', 'amzn', 'brk', 'b']"
3,2022-12-30,20:29:20,ItsJennyJ,0,Twitter for Android,@Apple I have an Apple ipod from 2012.\nI'd li...,0.000000,0.000000,Neutral,0.0,...,0,"{'! count': 0, '"" count': 0, '# count': 2, '$ ...",4.705882,6.8,0.794118,0.411765,i have an apple ipod from \ni d like to tar...,"['i', 'have', 'an', 'apple', 'ipod', 'from', '...",apple ipod like tart u ing apple id order tart...,"['apple', 'ipod', 'like', 'tart', 'u', 'ing', ..."
4,2022-12-30,20:29:11,LlcBillionaire,0,Twitter Web App,The biggest â and maybe the best â financi...,0.150000,0.500000,Positive,1.0,...,0,"{'! count': 0, '"" count': 0, '# count': 0, '$ ...",5.161290,31.0,0.903226,0.419355,the bigge t â and maybe the be t â financi...,"['the', 'bigge', 't', 'â', 'and', 'maybe', 'th...",bigge â maybe â financial olution hould u ...,"['bigge', 'â\x80\x94', 'maybe', 'â\x80\x94', '..."


## 4.3 Word2Vec

In [23]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

LogisticRegression()

In [24]:
predicted_y_test = classifier.predict(X_test)
print("Logistic Regression Accuracy:", metrics.accuracy_score(y_test, predicted_y_test))
print("Logistic Regression Precision:", metrics.precision_score(y_test, predicted_y_test, average='micro'))
print("Logistic Regression Recall:", metrics.recall_score(y_test, predicted_y_test, average='micro'))

Logistic Regression Accuracy: 0.637923531240286
Logistic Regression Precision: 0.637923531240286
Logistic Regression Recall: 0.637923531240286


### 4.4 Stock